In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from utils.utils import zscore_transform, zscore_inverse_transform, get_tsc_train_dataset, check_normalization_quality

In [3]:
iawe_label_dict = {
    "air_conditioner_1": 0,
    "air_conditioner_2": 1,
    "computer": 2,
    "fridges": 3,
    "television": 4,
    "washing_machine": 5
}
# Load the dataset
data_path = "<path_to_dataset>/iAWE/train_test_np"
# Load the dataset
X_train = np.load(f"{data_path}/X_train.npy")
X_test = np.load(f"{data_path}/X_test.npy")
y_train = np.load(f"{data_path}/y_train.npy")
y_test = np.load(f"{data_path}/y_test.npy")
X_train = X_train[:, :, [0, 1, 3, 4]]
X_test = X_test[:, :, [0, 1, 3, 4]]
X_total = np.concatenate((X_train, X_test), axis=0)
y_total = np.concatenate((y_train, y_test), axis=0)

In [21]:
print(f"Total data shape: {X_total.shape}, Labels shape: {y_total.shape}")

Total data shape: (3936614, 120, 4), Labels shape: (3936614,)


In [22]:
import numpy as np
import os

def zscore_transform(data, save_path="zscore_params.npz"):
    """
    Apply Z-score normalization to 3D data and save parameters
    
    Args:
        data: 3D numpy array with shape (n_samples, seq_len, n_features)
        save_path: Path to save mean/std parameters
    
    Returns:
        normalized_data: Z-score normalized data with same shape as input
    """
    print(f"Input data shape: {data.shape}")
    
    # Calculate mean and std across samples and time steps for each feature
    # Shape: (n_features,)
    mean_vals = data.mean(axis=(0, 1))
    std_vals = data.std(axis=(0, 1))
    
    print(f"Mean values per feature: {mean_vals}")
    print(f"Std values per feature: {std_vals}")
    
    # Avoid division by zero (if std is 0, keep original values)
    std_vals[std_vals == 0] = 1.0
    
    # Z-score normalization: (x - mean) / std
    normalized_data = (data - mean_vals) / std_vals
    
    # Save parameters
    np.savez(save_path, mean_vals=mean_vals, std_vals=std_vals)
    print(f"Z-score parameters saved to: {save_path}")
    
    print(f"Normalized data mean: {normalized_data.mean(axis=(0,1))}")
    print(f"Normalized data std: {normalized_data.std(axis=(0,1))}")
    print(f"Normalized data range: [{normalized_data.min():.6f}, {normalized_data.max():.6f}]")
    
    return normalized_data

def zscore_inverse_transform(normalized_data, params_path="zscore_params.npz"):
    """
    Inverse Z-score normalization using saved parameters
    
    Args:
        normalized_data: Normalized 3D numpy array with shape (n_samples, seq_len, n_features)
        params_path: Path to load mean/std parameters
    
    Returns:
        original_data: Data in original scale
    """
    print(f"Normalized data shape: {normalized_data.shape}")
    
    # Load parameters
    if not os.path.exists(params_path):
        raise FileNotFoundError(f"Parameters file not found: {params_path}")
    
    params = np.load(params_path)
    mean_vals = params['mean_vals']
    std_vals = params['std_vals']
    
    print(f"Loaded mean values: {mean_vals}")
    print(f"Loaded std values: {std_vals}")
    
    # Inverse transform: normalized_data * std + mean
    original_data = normalized_data * std_vals + mean_vals
    
    print(f"Inverse transformed data range: [{original_data.min():.6f}, {original_data.max():.6f}]")
    
    return original_data

# Utility function to check normalization quality
def check_normalization_quality(original_data, normalized_data, params_path):
    """
    Check the quality of normalization
    """
    # Load parameters
    params = np.load(params_path)
    mean_vals = params['mean_vals']
    std_vals = params['std_vals']
    
    # Check if normalized data has mean~0 and std~1
    actual_mean = normalized_data.mean(axis=(0, 1))
    actual_std = normalized_data.std(axis=(0, 1))
    
    print("=== Normalization Quality Check ===")
    print(f"Expected mean: {np.zeros_like(mean_vals)}")
    print(f"Actual mean: {actual_mean}")
    print(f"Mean error: {np.abs(actual_mean).max():.10f}")
    
    print(f"\nExpected std: {np.ones_like(std_vals)}")
    print(f"Actual std: {actual_std}")
    print(f"Std error: {np.abs(actual_std - 1).max():.10f}")
    
    # Test inverse transform
    reconstructed = zscore_inverse_transform(normalized_data, params_path)
    reconstruction_error = np.mean(np.abs(original_data - reconstructed))
    print(f"\nReconstruction error: {reconstruction_error:.10f}")
    
    return actual_mean, actual_std, reconstruction_error

In [27]:
data_path = f"<path>/datasets/NILMArchive_2025/iAWE/utils/zscore_params.npz"

In [ ]:
X_total_scaled = zscore_transform(X_total, save_path=data_path)

In [28]:
X_total = zscore_inverse_transform(X_total_scaled, params_path=data_path)
# Check normalization quality
check_normalization_quality(X_total, X_total_scaled, params_path=data_path)

Normalized data shape: (3936614, 120, 4)
Loaded mean values: [  1.23506228 252.51523983  48.02349322 274.46676774]
Loaded std values: [  2.64973896 517.86262007 122.20907464 523.29961379]
Inverse transformed data range: [-646.482000, 3986.751000]
=== Normalization Quality Check ===
Expected mean: [0. 0. 0. 0.]
Actual mean: [-5.88201042e-16  1.04817923e-15  4.74763512e-16  5.00143392e-15]
Mean error: 0.0000000000

Expected std: [1. 1. 1. 1.]
Actual std: [1. 1. 1. 1.]
Std error: 0.0000000000
Normalized data shape: (3936614, 120, 4)
Loaded mean values: [  1.23506228 252.51523983  48.02349322 274.46676774]
Loaded std values: [  2.64973896 517.86262007 122.20907464 523.29961379]
Inverse transformed data range: [-646.482000, 3986.751000]

Reconstruction error: 0.0000000000


(array([-5.88201042e-16,  1.04817923e-15,  4.74763512e-16,  5.00143392e-15]),
 array([1., 1., 1., 1.]),
 np.float64(0.0))

In [32]:
# Split the scaled data into different sets
from sklearn.model_selection import train_test_split
X_sp_scaled, X_users_scaled, y_sp_scaled, y_users_scaled = train_test_split(
    X_total_scaled, y_total, test_size=0.4, random_state=42, stratify=y_total
)
X_other_users_scaled, X_atk_scaled, y_other_users_scaled, y_atk_scaled = train_test_split(
    X_users_scaled, y_users_scaled, test_size=0.2, random_state=42, stratify=y_users_scaled
)

print(f"X_sp_scaled shape: {X_sp_scaled.shape}, y_sp_scaled shape: {y_sp_scaled.shape}")
print(f"X_other_users_scaled shape: {X_other_users_scaled.shape}, y_other_users_scaled shape: {y_other_users_scaled.shape}")
print(f"X_atk_scaled shape: {X_atk_scaled.shape}, y_atk_scaled shape: {y_atk_scaled.shape}")

X_sp_scaled shape: (2361968, 120, 4), y_sp_scaled shape: (2361968,)
X_other_users_scaled shape: (1259716, 120, 4), y_other_users_scaled shape: (1259716,)
X_atk_scaled shape: (314930, 120, 4), y_atk_scaled shape: (314930,)


In [33]:
print(np.unique(y_sp_scaled, return_counts=True))
print(np.unique(y_other_users_scaled, return_counts=True))
print(np.unique(y_atk_scaled, return_counts=True))

(array([0, 1, 2, 3, 4, 5]), array([ 164729,  183315,  730211, 1109255,  171118,    3340]))
(array([0, 1, 2, 3, 4, 5]), array([ 87856,  97768, 389445, 591603,  91263,   1781]))
(array([0, 1, 2, 3, 4, 5]), array([ 21964,  24442,  97362, 147901,  22816,    445]))


In [36]:
X_sp_train, X_sp_test, y_sp_train, y_sp_test = train_test_split(
    X_sp_scaled, y_sp_scaled, test_size=0.1, random_state=42, stratify=y_sp_scaled
)
print(f"X_sp_train shape: {X_sp_train.shape}, y_sp_train shape: {y_sp_train.shape}")
print(f"X_sp_test shape: {X_sp_test.shape}, y_sp_test shape: {y_sp_test.shape}")

X_atk_train, X_atk_test, y_atk_train, y_atk_test = train_test_split(
    X_atk_scaled, y_atk_scaled, test_size=0.1, random_state=42, stratify=y_atk_scaled
)
print(f"X_atk_train shape: {X_atk_train.shape}, y_atk_train shape: {y_atk_train.shape}")
print(f"X_atk_test shape: {X_atk_test.shape}, y_atk_test shape: {y_atk_test.shape}")

X_atk_train shape: (283437, 120, 4), y_atk_train shape: (283437,)
X_atk_test shape: (31493, 120, 4), y_atk_test shape: (31493,)


In [37]:
# Save the data
data_path = f"<path>/datasets/NILMArchive_2025/iAWE/scaled_train_test_np"
np.save(f"{data_path}/X_sp_train.npy", X_sp_train)
np.save(f"{data_path}/y_sp_train.npy", y_sp_train)
np.save(f"{data_path}/X_sp_test.npy", X_sp_test)
np.save(f"{data_path}/y_sp_test.npy", y_sp_test)
np.save(f"{data_path}/X_other_users_scaled.npy", X_other_users_scaled)
np.save(f"{data_path}/y_other_users_scaled.npy", y_other_users_scaled)
np.save(f"{data_path}/X_atk_train.npy", X_atk_train)
np.save(f"{data_path}/y_atk_train.npy", y_atk_train)
np.save(f"{data_path}/X_atk_test.npy", X_atk_test)
np.save(f"{data_path}/y_atk_test.npy", y_atk_test)

In [5]:
x_train, y_train, x_test, y_test = get_tsc_train_dataset("iAWE", "sp")
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

x_train, y_train, x_test, y_test = get_tsc_train_dataset("iAWE", "atk")
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(2125771, 120, 4) (2125771,) (236197, 120, 4) (236197,)
(283437, 120, 4) (283437,) (31493, 120, 4) (31493,)


# Do other stuff with the data

In [ ]:
def plot_class_distribution(y_data, figsize=(12, 6), title=None):
    """
    Create a bar chart showing the distribution of samples across classes.
    
    Args:
        y_data: Array containing class labels
        figsize: Figure size as tuple (width, height)
        title: Optional title for the plot. If None, a default title is used.
        
    Returns:
        None (displays the plot)
    """    
    # Count the number of samples for each class
    unique_labels, counts = np.unique(y_data, return_counts=True)
    
    # Create a bar chart
    plt.figure(figsize=figsize)
    bars = plt.bar(
        [f"Class {int(label)}" for label in unique_labels], 
        counts,
        color='skyblue'
    )
    
    # Add the actual count on top of each bar
    for bar, count in zip(bars, counts):
        plt.text(
            bar.get_x() + bar.get_width()/2,
            bar.get_height() + 10,
            count,
            ha='center',
            fontweight='bold'
        )
    
    # Set the plot title
    if title is None:
        title = f'Distribution of Samples Across Classes (n={len(y_data)})'
    plt.title(title)
    
    plt.xlabel('Class')
    plt.ylabel('Number of Samples')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()
    
    # Print class distribution statistics
    print("Class distribution statistics:")
    for label, count in zip(unique_labels, counts):
        percentage = (count / len(y_data)) * 100
        print(f"Class {int(label)}: {count} samples ({percentage:.2f}%)")

# Plot the training data distribution
plot_class_distribution(y_train, title='Distribution of Training Samples Across Classes')

In [ ]:
import time

# Set a random seed based on current time for different selections each run
np.random.seed(int(time.time()))

# Create a figure with 6 subplots (one for each class)
fig, axes = plt.subplots(len(iawe_label_dict), 1, figsize=(15, 12), sharex=True)

# Reverse the label_dict to map from numbers to labels
label_names = {v: k for k, v in iawe_label_dict.items()}

# Find one sample for each class
class_samples = {}
for idx, label in enumerate(y_train):
    prob = np.random.rand()
    if label not in class_samples:
        if prob > 0.5:  # 10% chance to select a sample
            class_samples[label] = idx
    if len(class_samples) == len(iawe_label_dict):
        break

# Get the selected indices - one from each class
selected_indices = list(class_samples.values())

# Plot one sample from each class
for i, label in enumerate(sorted(class_samples.keys())):
    idx = class_samples[label]
    axes[i].plot(X_train_Irms[idx], 'b-')
    # Use the label dictionary to get the actual name of the device
    class_name = label_names.get(label, f"Unknown ({label})")
    axes[i].set_title(f'Sample {idx}, Class: {class_name}')
    axes[i].set_ylabel('Current (Irms)')
    axes[i].grid(True)

# Add a common x-axis label
plt.xlabel('Time step')
plt.suptitle('Current (Irms) samples - One from each class')
plt.tight_layout()
plt.show()

print("Class samples chosen for this run:", class_samples)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
n = 20              # Number of data points
sampling_rate = 1.0 # Hz (samples per second)
sample_spacing = 1.0 / sampling_rate # seconds (time between samples)

# Create a sample signal (e.g., a sine wave at 0.2 Hz + 0.4 Hz)
# Frequencies present should be less than Nyquist frequency (0.5 Hz)
time_vector = np.arange(n) * sample_spacing
signal = (np.sin(2 * np.pi * 0.2 * time_vector) +
          0.5 * np.sin(2 * np.pi * 0.4 * time_vector) +
          0.1 * np.random.randn(n)) # Adding some noise

# 1. Compute the FFT using np.fft.fft
fft_coefficients = np.fft.fft(signal)

# 2. Compute the frequencies using np.fft.fftfreq
frequencies = np.fft.fftfreq(n, d=sample_spacing)

# 3. Determine the number of unique points in the FFT for a real signal
# This corresponds to 0 Hz up to and including the Nyquist frequency
num_unique_points = n // 2 + 1

# 4. Slice the FFT_coefficients and frequencies to get the positive spectrum
# (0 Hz to Nyquist frequency)
fft_positive_spectrum = fft_coefficients[:num_unique_points]
frequencies_positive_spectrum = frequencies[:num_unique_points]

# The magnitudes of the FFT coefficients
fft_magnitudes = np.abs(fft_positive_spectrum)

# Output the results
print(f"Number of data points (n): {n}")
print(f"Sampling rate: {sampling_rate} Hz")
print(f"Nyquist frequency: {sampling_rate / 2} Hz")
print("\nFrequencies from np.fft.fftfreq (0 to Nyquist range):")
for i in range(num_unique_points):
    # For display, we often show the Nyquist frequency as positive
    display_freq = frequencies_positive_spectrum[i]
    if i == n // 2 and n % 2 == 0: # Nyquist frequency for even n
        print(f"  Bin {i}: {display_freq:.4f} Hz (Nyquist, magnitude: {np.abs(display_freq):.4f} Hz) - FFT Magnitude: {fft_magnitudes[i]:.4f}")
    else:
        print(f"  Bin {i}: {display_freq:.4f} Hz - FFT Magnitude: {fft_magnitudes[i]:.4f}")


# Plotting the one-sided spectrum (0 to Nyquist)
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(time_vector, signal)
plt.title("Original Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")

plt.subplot(1, 2, 2)
# For plotting, it's conventional to use the positive Nyquist frequency label
plot_frequencies = np.abs(frequencies_positive_spectrum) if n % 2 == 0 else frequencies_positive_spectrum
# Ensure the Nyquist frequency is positive for plotting if it was negative
if n % 2 == 0 and frequencies_positive_spectrum[-1] < 0:
    plot_frequencies[-1] = -frequencies_positive_spectrum[-1]

plt.stem(plot_frequencies, fft_magnitudes)
plt.title("One-Sided Magnitude Spectrum (0 to Nyquist)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.xlim(0, sampling_rate / 2) # Show up to Nyquist
plt.grid(True)

plt.tight_layout()
plt.show()

print("\nNote on np.fft.fftfreq output for Nyquist frequency:")
print("For an even number of samples 'n', the frequency at index n//2 in the output of")
print("np.fft.fftfreq(n, d) is -(sampling_rate / 2). This bin still corresponds to")
print("the Nyquist frequency magnitude of (sampling_rate / 2).")

In [ ]:
def plot_frequency_domain_analysis(time_series_data, sample_indices, title, sample_rate=1.0, figsize=(14, 10)):
    """
    Perform frequency domain analysis using DFT for selected samples, displaying with bar charts.
    
    Args:
        time_series_data: The time series data (2D array, samples x time steps)
        sample_indices: Indices of samples to analyze
        title: Title for the plot
        sample_rate: Sampling rate (default=1.0 Hz)
        figsize: Size of the figure
    """
    n_samples = len(sample_indices)
    fig, axes = plt.subplots(n_samples, 2, figsize=figsize)
    
    for i, idx in enumerate(sample_indices):
        # Get the sample
        sample = time_series_data[idx]
        n = len(sample)
        
        # Perform DFT using FFT
        fft_result = np.fft.fft(sample)
        magnitude = np.abs(fft_result)[:n//2]  # Take only the first half (positive frequencies)
        
        # Calculate frequencies
        freq = np.fft.fftfreq(n, d=1/sample_rate)[:n//2]
        
        # Plot time domain signal
        axes[i, 0].plot(sample)
        axes[i, 0].set_title(f'Sample {idx} - Time Domain')
        axes[i, 0].set_xlabel('Time Step')
        axes[i, 0].set_ylabel('Amplitude')
        axes[i, 0].grid(True)
        
        # Plot frequency domain as bar chart
        axes[i, 1].bar(freq, magnitude, width=freq[1]-freq[0] if len(freq) > 1 else 0.1)
        axes[i, 1].set_title(f'Sample {idx} - Frequency Domain')
        axes[i, 1].set_xlabel('Frequency')
        axes[i, 1].set_ylabel('Magnitude')
        axes[i, 1].grid(True)
        
        # Get class label name for this sample (if available)
        if 'label_names' in globals() and 'y_train' in globals():
            class_id = y_train[idx]
            class_name = label_names.get(class_id, f"Class {class_id}")
            axes[i, 0].set_title(f'Sample {idx} - {class_name} - Time Domain')
            axes[i, 1].set_title(f'Sample {idx} - {class_name} - Frequency Domain')
    
    plt.suptitle(title)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot frequency domain for different datasets
# First for electrical current data (one from each class)
plot_frequency_domain_analysis(
    X_train_Irms, 
    [class_samples[label] for label in sorted(class_samples.keys())[:3]], 
    'Frequency Domain Analysis of Current (Irms) - First 3 Classes', 
    sample_rate=0.5
)

# Second plot for the remaining 3 classes
plot_frequency_domain_analysis(
    X_train_Irms, 
    [class_samples[label] for label in sorted(class_samples.keys())[3:]], 
    'Frequency Domain Analysis of Current (Irms) - Last 3 Classes', 
    sample_rate=0.5
)

In [ ]:
def modify_frequency_and_inverse_transform(signal, target_freq=0.3, frequency_amplification=10.0, sample_rate=1.0, plot=True):
    """
    Modify the magnitude at a specific frequency and perform inverse Fourier transform
    
    Args:
        signal: Input time series signal (1D array)
        target_freq: Target frequency to amplify (default=0.3)
        frequency_amplification: Amount to multiply the magnitude by (default=10.0)
        sample_rate: Sampling rate of signal (default=1.0 Hz)
        plot: Whether to plot the results (default=True)
    
    Returns:
        Modified time domain signal
    """
    # Perform FFT
    n = len(signal)
    fft_result = np.fft.fft(signal)
    magnitude = np.abs(fft_result)
    phase = np.angle(fft_result)
    
    # Calculate frequency bins
    freq = np.fft.fftfreq(n, d=1/sample_rate)
    
    # Find the index closest to the target frequency
    target_idx = np.argmin(np.abs(freq - target_freq))
    conjugate_idx = n - target_idx if target_idx > 0 else 0
    
    # Store the original magnitude at the target frequency
    original_magnitude = magnitude[target_idx]
    
    # Modify the magnitude at the target frequency and its conjugate
    fft_result[target_idx] *= frequency_amplification
    if target_idx != conjugate_idx:  # Ensure we maintain complex conjugate symmetry
        fft_result[conjugate_idx] *= frequency_amplification
    
    # Inverse FFT to get back to time domain
    modified_signal = np.real(np.fft.ifft(fft_result))
    
    # Calculate the difference between original and modified signals
    difference = signal - modified_signal 
    
    if plot:
        # Plot the original and modified signals
        fig, axes = plt.subplots(3, 2, figsize=(14, 12))
        
        # Plot original signal in time domain
        axes[0, 0].plot(signal)
        axes[0, 0].set_title('Original Signal (Time Domain)')
        axes[0, 0].set_xlabel('Time Step')
        axes[0, 0].set_ylabel('Amplitude')
        axes[0, 0].grid(True)
        
        # Plot original signal in frequency domain as bar chart
        orig_fft = np.fft.fft(signal)
        orig_magnitude = np.abs(orig_fft)[:n//2]
        orig_freq = freq[:n//2]
        axes[0, 1].bar(orig_freq, orig_magnitude, width=orig_freq[1]-orig_freq[0] if len(orig_freq) > 1 else 0.1)
        axes[0, 1].set_title('Original Signal (Frequency Domain)')
        axes[0, 1].set_xlabel('Frequency')
        axes[0, 1].set_ylabel('Magnitude')
        axes[0, 1].grid(True)
        axes[0, 1].axvline(x=freq[target_idx], color='r', linestyle='--', 
                       label=f'Target: {freq[target_idx]:.3f} Hz (Mag: {original_magnitude:.3f})')
        axes[0, 1].legend()
        
        # Plot modified signal in time domain
        axes[1, 0].plot(modified_signal)
        axes[1, 0].set_title('Modified Signal (Time Domain)')
        axes[1, 0].set_xlabel('Time Step')
        axes[1, 0].set_ylabel('Amplitude')
        axes[1, 0].grid(True)
        
        # Plot modified signal in frequency domain as bar chart
        mod_magnitude = np.abs(fft_result)[:n//2]
        axes[1, 1].bar(orig_freq, mod_magnitude, width=orig_freq[1]-orig_freq[0] if len(orig_freq) > 1 else 0.1)
        axes[1, 1].set_title('Modified Signal (Frequency Domain)')
        axes[1, 1].set_xlabel('Frequency')
        axes[1, 1].set_ylabel('Magnitude')
        axes[1, 1].grid(True)
        axes[1, 1].axvline(x=freq[target_idx], color='r', linestyle='--',
                       label=f'Amplified: {freq[target_idx]:.3f} Hz (Mag: {mod_magnitude[target_idx%len(mod_magnitude)]:.3f})')
        axes[1, 1].legend()
        
        # Plot the difference between original and modified signals
        axes[2, 0].plot(difference)
        axes[2, 0].set_title('Difference (Modified - Original)')
        axes[2, 0].set_xlabel('Time Step')
        axes[2, 0].set_ylabel('Amplitude')
        axes[2, 0].grid(True)
        
        # Plot the difference in frequency domain
        diff_fft = np.fft.fft(difference)
        diff_magnitude = np.abs(diff_fft)[:n//2]
        axes[2, 1].bar(orig_freq, diff_magnitude, width=orig_freq[1]-orig_freq[0] if len(orig_freq) > 1 else 0.1)
        axes[2, 1].set_title('Difference (Frequency Domain)')
        axes[2, 1].set_xlabel('Frequency')
        axes[2, 1].set_ylabel('Magnitude')
        axes[2, 1].grid(True)
        axes[2, 1].axvline(x=freq[target_idx], color='r', linestyle='--',
                       label=f'Difference at {freq[target_idx]:.3f} Hz')
        axes[2, 1].legend()
        
        plt.tight_layout()
        plt.suptitle(f'Frequency {freq[target_idx]:.3f} Hz amplified by {frequency_amplification}x\n'
                    f'Original magnitude: {original_magnitude:.3f}, Modified magnitude: {mod_magnitude[target_idx%len(mod_magnitude)]:.3f}')
        plt.subplots_adjust(top=0.92)
        plt.show()
    
    return modified_signal

# Example usage with a sample from the dataset
sample_idx = class_samples[1]  # Using a television sample
sample_signal = X_train_Irms[sample_idx]

# Apply the function with different amplification values
k_value = 1.2  # You can change this to any amplification factor you want
modified_signal = modify_frequency_and_inverse_transform(
    sample_signal, 
    target_freq=0.05, 
    frequency_amplification=k_value,
    sample_rate=1  # Using the same sample rate as in previous plots
)

In [ ]:
def visualize_signal_phase(signal, sample_rate=1.0, figsize=(14, 10)):
    """
    Visualize a signal in time domain, frequency domain (magnitude), and phase spectrum
    
    Args:
        signal: Input time series signal (1D array)
        sample_rate: Sampling rate of signal (default=1.0 Hz)
        figsize: Figure size (width, height)
    """
    # Perform FFT
    n = len(signal)
    fft_result = np.fft.fft(signal)
    magnitude = np.abs(fft_result)[:n//2]  # Take only the first half (positive frequencies)
    phase = np.angle(fft_result)[:n//2]    # Phase in radians
    
    # Calculate frequency bins (x-axis for frequency domain plots)
    freq = np.fft.fftfreq(n, d=1/sample_rate)[:n//2]
    
    # Create figure with 3 subplots
    fig, axes = plt.subplots(3, 1, figsize=figsize)
    
    # Plot time domain signal
    axes[0].plot(signal)
    axes[0].set_title('Time Domain Signal')
    axes[0].set_xlabel('Time Steps')
    axes[0].set_ylabel('Amplitude')
    axes[0].grid(True)
    
    # Plot magnitude spectrum
    axes[1].bar(freq, magnitude, width=freq[1]-freq[0] if len(freq) > 1 else 0.1)
    axes[1].set_title('Frequency Domain - Magnitude Spectrum')
    axes[1].set_xlabel('Frequency (Hz)')
    axes[1].set_ylabel('Magnitude')
    axes[1].grid(True)
    
    # Plot phase spectrum
    axes[2].bar(freq, phase, width=freq[1]-freq[0] if len(freq) > 1 else 0.1)
    axes[2].set_title('Frequency Domain - Phase Spectrum (radians)')
    axes[2].set_xlabel('Frequency (Hz)')
    axes[2].set_ylabel('Phase (radians)')
    axes[2].set_ylim(-np.pi, np.pi)  # Phase is typically in range [-π, π]
    axes[2].grid(True)
    
    # Add horizontal lines at -π, 0, and π for reference
    axes[2].axhline(y=0, color='r', linestyle='-', alpha=0.3)
    axes[2].axhline(y=np.pi, color='g', linestyle='--', alpha=0.3)
    axes[2].axhline(y=-np.pi, color='g', linestyle='--', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return fig, axes

# Visualize the phase for a sample signal
sample_idx = class_samples[1]  # Using the same air_conditioner_2 sample
sample_signal = X_train_Irms[sample_idx]

# Call the function with appropriate sample rate
# visualize_signal_phase(sample_signal, sample_rate=0.5)

def modify_signal_phase(signal, target_freq=0.05, phase_shift=np.pi/2, sample_rate=1.0, plot=True):
    """
    Modify the phase at a specific frequency and perform inverse Fourier transform
    
    Args:
        signal: Input time series signal (1D array)
        target_freq: Target frequency to modify the phase (default=0.05)
        phase_shift: Amount to shift the phase by (default=π/2 radians)
        sample_rate: Sampling rate of signal (default=1.0 Hz)
        plot: Whether to plot the results (default=True)
    
    Returns:
        Modified time domain signal
    """
    # Perform FFT
    n = len(signal)
    fft_result = np.fft.fft(signal)
    magnitude = np.abs(fft_result)
    phase = np.angle(fft_result)
    
    # Calculate frequency bins
    freq = np.fft.fftfreq(n, d=1/sample_rate)
    
    # Find the index closest to the target frequency
    target_idx = np.argmin(np.abs(freq - target_freq))
    conjugate_idx = n - target_idx if target_idx > 0 else 0
    
    # Store the original phase at the target frequency
    original_phase = phase[target_idx]
    
    # Create a copy of the FFT result to modify
    modified_fft = fft_result.copy()
    
    # Modify the phase at the target frequency and its conjugate
    new_phase = original_phase + phase_shift
    
    # Convert magnitude and new phase back to complex form (using Euler's formula)
    modified_fft[target_idx] = magnitude[target_idx] * np.exp(1j * new_phase)
    if target_idx != conjugate_idx:  # Ensure we maintain complex conjugate symmetry
        modified_fft[conjugate_idx] = magnitude[conjugate_idx] * np.exp(-1j * new_phase)
    
    # Inverse FFT to get back to time domain
    modified_signal = np.real(np.fft.ifft(modified_fft))
    
    if plot:
        # Create a figure with 3x2 subplots
        fig, axes = plt.subplots(3, 2, figsize=(15, 12))
        
        # Plot original and modified signals in time domain
        axes[0, 0].plot(signal)
        axes[0, 0].set_title('Original Signal (Time Domain)')
        axes[0, 0].set_xlabel('Time Steps')
        axes[0, 0].set_ylabel('Amplitude')
        axes[0, 0].grid(True)
        
        axes[0, 1].plot(modified_signal)
        axes[0, 1].set_title('Phase-Modified Signal (Time Domain)')
        axes[0, 1].set_xlabel('Time Steps')
        axes[0, 1].set_ylabel('Amplitude')
        axes[0, 1].grid(True)
        
        # Plot original and modified magnitude spectrum
        orig_freq = freq[:n//2]
        orig_magnitude = magnitude[:n//2]
        
        axes[1, 0].bar(orig_freq, orig_magnitude, width=orig_freq[1]-orig_freq[0] if len(orig_freq) > 1 else 0.1)
        axes[1, 0].set_title('Original Magnitude Spectrum')
        axes[1, 0].set_xlabel('Frequency (Hz)')
        axes[1, 0].set_ylabel('Magnitude')
        axes[1, 0].grid(True)
        
        mod_magnitude = np.abs(modified_fft)[:n//2]
        axes[1, 1].bar(orig_freq, mod_magnitude, width=orig_freq[1]-orig_freq[0] if len(orig_freq) > 1 else 0.1)
        axes[1, 1].set_title('Modified Magnitude Spectrum')
        axes[1, 1].set_xlabel('Frequency (Hz)')
        axes[1, 1].set_ylabel('Magnitude')
        axes[1, 1].grid(True)
        
        # Plot original and modified phase spectrum
        orig_phase = np.angle(fft_result)[:n//2]
        axes[2, 0].bar(orig_freq, orig_phase, width=orig_freq[1]-orig_freq[0] if len(orig_freq) > 1 else 0.1)
        axes[2, 0].set_title('Original Phase Spectrum')
        axes[2, 0].set_xlabel('Frequency (Hz)')
        axes[2, 0].set_ylabel('Phase (radians)')
        axes[2, 0].set_ylim(-np.pi, np.pi)
        axes[2, 0].grid(True)
        axes[2, 0].axvline(x=freq[target_idx], color='r', linestyle='--',
                       label=f'Target: {freq[target_idx]:.3f} Hz (Phase: {original_phase:.2f} rad)')
        axes[2, 0].legend()
        
        mod_phase = np.angle(modified_fft)[:n//2]
        axes[2, 1].bar(orig_freq, mod_phase, width=orig_freq[1]-orig_freq[0] if len(orig_freq) > 1 else 0.1)
        axes[2, 1].set_title('Modified Phase Spectrum')
        axes[2, 1].set_xlabel('Frequency (Hz)')
        axes[2, 1].set_ylabel('Phase (radians)')
        axes[2, 1].set_ylim(-np.pi, np.pi)
        axes[2, 1].grid(True)
        axes[2, 1].axvline(x=freq[target_idx], color='r', linestyle='--',
                       label=f'Modified: {freq[target_idx]:.3f} Hz (Phase: {new_phase:.2f} rad)')
        axes[2, 1].legend()
        
        plt.tight_layout()
        plt.suptitle(f'Phase modification at {freq[target_idx]:.3f} Hz by {phase_shift:.2f} radians\n'
                    f'Original phase: {original_phase:.2f} rad, Modified phase: {new_phase:.2f} rad')
        plt.subplots_adjust(top=0.92)
        plt.show()
    
    return modified_signal

# Apply the phase modification to the same sample
sample_idx = class_samples[1]  # Using the air_conditioner_2 sample again
sample_signal = X_train_Irms[sample_idx]

# Try different phase shifts
phase_shifts = [np.pi/4]
for shift in phase_shifts:
    phase_modified_signal = modify_signal_phase(
        sample_signal, 
        target_freq=0.05,  # Target a specific frequency
        phase_shift=shift,
        sample_rate=0.5    # Using the same sample rate as before
    )
    print(f"Phase shift of {shift:.2f} radians applied")